In [1]:
import orekit
import numpy as np
from mistune import markdown

vm = orekit.initVM()
from orekit.pyhelpers import setup_orekit_curdir, absolutedate_to_datetime, datetime_to_absolutedate
setup_orekit_curdir(filename = '../orekit_examples/orekit-data-master')
from datetime import datetime
import json

In [2]:
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator
from org.orekit.orbits import KeplerianOrbit, OrbitType, PositionAngleType
from org.orekit.utils import Constants
from org.orekit.time import AbsoluteDate
from math import degrees
mu = Constants.WGS84_EARTH_MU

def pv_to_numpy(pv_object):
    # Extract the Position and Velocity Vector3D objects
    pos = pv_object.getPosition()
    vel = pv_object.getVelocity()

    # Create the numpy array
    return np.array([
        pos.getX(), pos.getY(), pos.getZ(),
        vel.getX(), vel.getY(), vel.getZ()
    ])



In [3]:
with open('orbit_files/orbital_elements_ltan14.kepl') as f:
    kepl = json.load(f)

# Construct a TLE from Mean Elements

TLE constructor that uses orekt. For this we just assume that osculating and mean elements are the same, and show how can we produce a TLE from the mean elements. Of course this can not be used in practice.


In [4]:
mean_n = (mu / (kepl['sma']*1000)**3 ) ** 0.5
mean_e = kepl['ecc']
mean_i = kepl['incl']
mean_raan = kepl['raan']
mean_pa = kepl['argP']
mean_m = kepl['MA']


new_tle_mean = TLE(
    99999, # number
    'U', # classification
    2026, # year
    0, # launch number
    '', # launch piece
    2, # ephemeris type
    0, # element number
    datetime_to_absolutedate(datetime.strptime(kepl['epoch'], '%Y-%m-%dT%H:%M:%S')),
    mean_n,   # Mean Motion
    0.0,      # Mean Motion 1st deriv (optional)
    0.0,      # Mean Motion 2nd deriv (optional)
    mean_e,   # Eccentricity
    mean_i,   # Inclination
    mean_pa,  # Arg of Perigee
    mean_raan, # RAAN
    mean_m,   # Mean Anomaly
    0, # revolution number
    0.0 # Bstar
)
print(f"{new_tle_mean.getLine1()}")
print(f"{new_tle_mean.getLine2()}")

1 99999U 26000    26088.00000000  .00000000  00000-0  00000-0 2    02
2 99999  97.4043  39.7161 0001000 269.2669  89.8890 15.20925629    05


# Fitting a TLE using a single Keplerian set of elements

We will use the above TLE, as an initial guess, for the Fit.

Moreover, we will degine a Keplerian orbit using the orbital elements we assume to be correct, and add this inital keplerian state to a java array list, that can be handled by Orekit.



In [5]:
from org.orekit.propagation import SpacecraftState
from org.orekit.frames import FramesFactory
from org.orekit.time import TimeScalesFactory
from java.util import ArrayList  # Import Java's ArrayList

inertial_frame = FramesFactory.getEME2000()
utc = TimeScalesFactory.getUTC()
mu = Constants.WGS84_EARTH_MU

# 2. Parse Epoch
# Orekit accepts ISO-8601 strings directly with a TimeScale
epoch = datetime_to_absolutedate(datetime.strptime(kepl['epoch'], '%Y-%m-%dT%H:%M:%S'))

# 3. Create Orbit
# Note: SMA converted from km to meters (x1000)
# Note: Angles assumed to be in Radians (as per typical JSON dumps)
orbit = KeplerianOrbit(
    kepl["sma"] * 1000.0,  # a (meters)
    kepl["ecc"],           # e
    kepl["incl"],          # i (radians)
    kepl["argP"],          # omega (radians)
    kepl["raan"],          # raan (radians)
    kepl["MA"],            # anomaly (radians)
    PositionAngleType.MEAN, # Specifies that 'MA' is Mean Anomaly
    inertial_frame,
    epoch,
    mu
)

state_at_epoch = SpacecraftState(orbit)
states_list = ArrayList()
states_list.add(state_at_epoch)

True

then continue in the most orekitish way, by TLE propagation builder, and the fitting algorithm

In [6]:
from org.orekit.propagation.analytical.tle.generation import FixedPointTleGenerationAlgorithm
from org.orekit.propagation.conversion import TLEPropagatorBuilder, FiniteDifferencePropagatorConverter

fpga = FixedPointTleGenerationAlgorithm()

tle_builder = TLEPropagatorBuilder(
    new_tle_mean,
    PositionAngleType.MEAN,
    1.0, # TODO, write what it is
    fpga
)
fitter = FiniteDifferencePropagatorConverter(tle_builder, 1.0, 10000)


Convert the fiter, so it matches the states list, (that is only one point at this stage.)

In [7]:
fitted_propagator_generic = fitter.convert(states_list, False, 'BSTAR')
fitted_tle_prop = TLEPropagator.cast_(fitted_propagator_generic)
fitted_tle = fitted_tle_prop.getTLE()

#
print(f"Generated from Fit:  {fitted_tle.getLine1()}")
print(f"                     {fitted_tle.getLine2()}")

Generated from Fit:  1 99999U 26000    26088.00000000  .00000000  00000-0  00000-0 2    02
                     2 99999  97.3171  40.0669 0012579 247.4676 111.8050 15.23054260    04


# Extracting Mean and Osculating Orbital Elements from TLE

initially we can check the TLEs, if the orbital elements were the mean ones, or the osculating ones. We will choose to proceed with the fitted TLE, so to check if we can extract back the orbital elements.

In [8]:
tle_line1 = '1 99999U 26000    26088.00000000  .00000000  00000-0  00000-0 2    02'
tle_line2 = '2 99999  97.4043  39.7161 0001000 269.2669   0.0000 15.20925629    03'

tle_line1 = '1 99999U 26000    26088.00000000  .00000000  00000-0  00000-0 2    02'
tle_line2 = '2 99999  97.3171  40.0669 0012579 247.4676 111.8050 15.23054260    04'
input_tle = TLE(tle_line1, tle_line2)

In [9]:
mean_n = input_tle.getMeanMotion()
mean_e = input_tle.getE()
mean_i = input_tle.getI()
mean_pa = input_tle.getPerigeeArgument()  # omega
mean_raan = input_tle.getRaan()           # Omega
mean_m = input_tle.getMeanAnomaly()       # M

mu = Constants.WGS84_EARTH_MU
mean_a = (mu / (mean_n ** 2)) ** (1.0/3.0)

print(f"--- Mean Elements (SGP4) ---")
print(f"Semi-major Axis (a): {mean_a / 1000:.3f} km")
print(f"Eccentricity (e):    {mean_e:.6f}")
print(f"Inclination (i):     {degrees(mean_i):.4f} deg")
print(f"Arg of Perigee (ω):  {degrees(mean_pa):.4f} deg")
print(f"RAAN (Ω):            {degrees(mean_raan):.4f} deg")
print(f"Mean Anomaly (M):    {degrees(mean_m):.4f} deg")

--- Mean Elements (SGP4) ---
Semi-major Axis (a): 6874.771 km
Eccentricity (e):    0.001258
Inclination (i):     97.3171 deg
Arg of Perigee (ω):  247.4676 deg
RAAN (Ω):            40.0669 deg
Mean Anomaly (M):    111.8050 deg


Get the osculating elements.

In [10]:
propagator = TLEPropagator.selectExtrapolator(input_tle)
state_at_epoch = propagator.getInitialState()
osc_orbit = OrbitType.KEPLERIAN.convertType(state_at_epoch.getOrbit())
osc_orbit = KeplerianOrbit(osc_orbit)

osc_a = osc_orbit.getA()
osc_e = osc_orbit.getE()
osc_i = osc_orbit.getI()
osc_pa = osc_orbit.getPerigeeArgument()
osc_raan =  osc_orbit.getRightAscensionOfAscendingNode()
osc_m = osc_orbit.getMeanAnomaly() # Or getTrueAnomaly()

print(f"\n--- Osculating Elements (Keplerian) ---")
print(f"Semi-major Axis (a): {osc_a / 1000:.3f} km")
print(f"Eccentricity (e):    {osc_e:.6f}")
print(f"Inclination (i):     {degrees(osc_i):.4f} deg")
print(f"Arg of Perigee (ω):  {degrees(osc_pa):.4f} deg")
print(f"RAAN (Ω):            {degrees(osc_raan):.4f} deg")
print(f"Mean Anomaly (M):    {degrees(osc_m):.4f} deg")


--- Osculating Elements (Keplerian) ---
Semi-major Axis (a): 6881.178 km
Eccentricity (e):    0.000100
Inclination (i):     97.3120 deg
Arg of Perigee (ω):  -91.1335 deg
RAAN (Ω):            40.0670 deg
Mean Anomaly (M):    90.4046 deg


# Conlusions, TODOs

We can immediatelly see, that the orbital elements do not match perfectly, something that is expected considering that the fit took place using only one point in the orbit, but it tried to optimize several parameters.

- This shows the need for more points.
- Moreover, we can also try to use Scipy optimize algorithms, or mcmc, for the TLE fitting, since the generation is pretty quick.